# Выбор локации для скважины

Для добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. И проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных:

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

Условия задачи:
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

### Импортируем нужные библиотеки.

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings('ignore')



### Извлекаем данные.

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

### Обзор данных.

#### data_1

In [3]:
display(data_1.duplicated().sum())
display(data_1.head())
data_1.info()

0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


#### data_2

In [4]:
display(data_2.duplicated().sum())
display(data_2.head())
data_2.info()

0

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


#### data_3

In [5]:
display(data_3.duplicated().sum())
display(data_3.head())
data_3.info()

0

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Предобработка данных.

#### Удаление неинформативных столбцов

In [6]:
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)
data_3 = data_3.drop(['id'], axis=1)

### Вывод.

В данных отстутствуют пропуски, дубликаты. Возможно это сгенерированные данные или они каким-то образом стандартизированы. Также удалили столбцы `id` т.к. они не понадобятся для построения модели.

## Обучение и проверка модели

Создадим функцию которая примет данные по одному региону, разделит данные на обучающую и валидационную выборки в соотношении 75:25, обучит модель и сделает предсказания на валидационной выборке, сохранит предсказания и правильные ответы на валидационной выборке, напечатает на экране средний запас предсказанного сырья и RMSE модели.

In [7]:
def full_lr(data):
    features = data.drop(['product'], axis=1)
    targett = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, targett,  test_size=0.25, random_state=12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions, index=features_valid.index)
    rmse = mean_squared_error(target_valid, predictions)**0.5
    mean_product = predictions.mean()
    print(' Cредний запас предсказанного сырья:', mean_product,'\n',
                 'RMSE модели линейной регрессии на валидационной выборке:', rmse)
    return (predictions, target_valid, mean_product, rmse)

In [8]:
data_1['product'].mean()

92.50000000000001

In [9]:
full_lr(data_1)

 Cредний запас предсказанного сырья: 92.59256778438035 
 RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


(71751     95.894952
 80493     77.572583
 2655      77.892640
 53233     90.175134
 91141     70.510088
             ...    
 12581    103.037104
 18456     85.403255
 73035     61.509833
 63834    118.180397
 43558    118.169392
 Length: 25000, dtype: float64,
 71751     10.038645
 80493    114.551489
 2655     132.603635
 53233    169.072125
 91141    122.325180
             ...    
 12581    170.116726
 18456     93.632175
 73035    127.352259
 63834     99.782700
 43558    177.821022
 Name: product, Length: 25000, dtype: float64,
 92.59256778438035,
 37.5794217150813)

In [10]:
full_lr(data_2)

 Cредний запас предсказанного сырья: 68.728546895446 
 RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617


(71751     82.663314
 80493     54.431786
 2655      29.748760
 53233     53.552133
 91141      1.243856
             ...    
 12581    136.869211
 18456    110.693465
 73035    137.879341
 63834     83.761966
 43558     53.958466
 Length: 25000, dtype: float64,
 71751     80.859783
 80493     53.906522
 2655      30.132364
 53233     53.906522
 91141      0.000000
             ...    
 12581    137.945408
 18456    110.992147
 73035    137.945408
 63834     84.038886
 43558     53.906522
 Name: product, Length: 25000, dtype: float64,
 68.728546895446,
 0.893099286775617)

In [11]:
full_lr(data_3)

 Cредний запас предсказанного сырья: 94.96504596800489 
 RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


(71751     93.599633
 80493     75.105159
 2655      90.066809
 53233    105.162375
 91141    115.303310
             ...    
 12581     78.765887
 18456     95.603394
 73035     99.407281
 63834     77.779912
 43558    129.032417
 Length: 25000, dtype: float64,
 71751     61.212375
 80493     41.850118
 2655      57.776581
 53233    100.053761
 91141    109.897122
             ...    
 12581     28.492402
 18456     21.431303
 73035    125.487229
 63834     99.422903
 43558    127.445075
 Name: product, Length: 25000, dtype: float64,
 94.96504596800489,
 40.02970873393434)

In [12]:
predictions_1, target_valid_1, mean_product_1, rmse_1 = full_lr(data_1)
predictions_2, target_valid_2, mean_product_2, rmse_2 = full_lr(data_2)
predictions_3, target_valid_3, mean_product_3, rmse_3 = full_lr(data_3)


 Cредний запас предсказанного сырья: 92.59256778438035 
 RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
 Cредний запас предсказанного сырья: 68.728546895446 
 RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617
 Cредний запас предсказанного сырья: 94.96504596800489 
 RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


Тут мы почитали средний запас предсказанного сырья - далее рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и сравним с этими значениями.

Среднеквадратичная ошибка показывает во втором регионе практически идеальный результат. Возможно синтетические данные специально синтезировали.

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраним в отдельных переменных.

In [13]:
BUDGET = 10000000000
BARR_1K = 450000
BARR = 450
HOLE = 200

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. 

In [14]:
barr_min = BUDGET / BARR_1K / HOLE
barr_min

111.11111111111111

In [15]:
product_mean = pd.DataFrame({'Cредний запас предсказанного сырья data_1': [mean_product_1],
                             'Cредний запас предсказанного сырья data_2':[mean_product_2],
                             'Cредний запас предсказанного сырья data_3': [mean_product_3],
                             'Объём сырья для безубыточной разработки':[barr_min]}).T

product_mean

,0
Cредний запас предсказанного сырья data_1,92.592568
Cредний запас предсказанного сырья data_2,68.728547
Cредний запас предсказанного сырья data_3,94.965046
Объём сырья для безубыточной разработки,111.111111


### Выводы по этапу подготовки расчёта прибыли.

Cредний запас предсказанного сырья не превышает объём сырья для безубыточной разработки. Чтобы выйти в плюс необходимо выбрать скважины с лучшими запасами.

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

* Выберем скважины с максимальными значениями предсказаний. 
* Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
* Рассчитаем прибыль для полученного объёма сырья.

In [16]:
def profit(target, predictions):
    predictions_top = predictions.sort_values(ascending=False) 
    predictions_top = predictions_top[:HOLE]
    top_200 = target[predictions_top.index].sum()
    predictions_profit = top_200 * BARR_1K - BUDGET
    return predictions_profit

In [17]:
max_profit = pd.DataFrame({'Прибыль для максимального объёма сырья data_1': [profit(target_valid_1, predictions_1)],
                           'Прибыль для максимального объёма сырья data_2': [profit(target_valid_2, predictions_2)],
                           'Прибыль для максимального объёма сырья data_3': [profit(target_valid_3, predictions_3)]}).T

max_profit

,0
Прибыль для максимального объёма сырья data_1,3.320826e+09
Прибыль для максимального объёма сырья data_2,2.415087e+09
Прибыль для максимального объёма сырья data_3,2.710350e+09


## Расчет рисков и прибыли для каждого региона:

* Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
* Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [18]:
def bootstap(target, probabilities):
    values = []
    prob = pd.Series(probabilities, index=target.index)
    state = np.random.RandomState(12345)
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = prob[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample))  
     
    values = pd.Series(values)
    mean = values.mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, mean, values.sem())
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = (values < 0).mean()
    
    print('Средняя прибыль: ', mean)
    print('95% доверительный интервал: ', lower, '-', upper)
    print ('Риск убытков:', risk)

In [19]:
bootstap(target_valid_1, predictions_1)

Средняя прибыль:  600735244.2611653
95% доверительный интервал:  12948331.135115242 - 1231163605.7914982
Риск убытков: 0.02


In [20]:
bootstap(target_valid_2, predictions_2)

Средняя прибыль:  665241058.2210723
95% доверительный интервал:  157988481.31991574 - 1197641587.4631522
Риск убытков: 0.003


In [21]:
bootstap(target_valid_3, predictions_3)

Средняя прибыль:  615559722.8409678
95% доверительный интервал:  -12218495.237293953 - 1230644473.941316
Риск убытков: 0.03


### Вывод

В данном проекте мы загрузили и провели обзор данных. Также провели предобработку - удаление первых столбцов в связи с их неинформативностью.

Разделили данные на тренировочную и валидационную выборки, провели обучение модели линейной регрессии.

Выяснили что средний запас предсказанного сырья не превышает объём сырья для безубыточной разработки. Чтобы выйти в плюс необходимо выбирать скважины с лучшими запасами.

Применили технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, среднюю прибыль, 95%-й доверительный интервал и риск убытков. 

Предложение, исходя из данных, выбрать для разработки месторождений второй регион т.к. он имеет самый высокий показатель средней прибыли и минимальный, из предложенных, риск убытков.